# CS4225/CS5425 Assignment 2

## 1. Data Preparation

In [0]:
import urllib
import zipfile
from pyspark.sql.types import *   
from pyspark.sql.functions import *
from pyspark.ml.feature import RFormula
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator



###Download and unzip citibike data from 2022/01 to 2023/07 into a dataframe

In [0]:
leftUrl = "https://s3.amazonaws.com/tripdata/"
rightUrl = "-citibike-tripdata.csv.zip"
leftPath = "/tmp/"
rightPath = "-citibike-tripdata.csv.zip"
yearAndMonths = [0] * 19
count = 202201

empRDD = spark.sparkContext.emptyRDD()
columns = StructType([StructField('ride_id', StringType(), True),
                      StructField('rideable_type', StringType(), True),
                      StructField('started_at', StringType(), True),
                      StructField('ended_at', StringType(), True),
                      StructField('start_station_name', StringType(), True),
                      StructField('start_station_id', StringType(), True),
                      StructField('end_station_name', StringType(), True),
                      StructField('end_station_id', StringType(), True),
                      StructField('start_lat', StringType(), True),
                      StructField('start_lng', StringType(), True),
                      StructField('end_lat', StringType(), True),
                      StructField('end_lng', StringType(), True),
                      StructField('member_casual', StringType(), True),]
)

bikeData = spark.createDataFrame(data=empRDD, schema=columns)

for i in range(0, 19):
    if str(count)[4:] == '13':
        count = 202301
    yearAndMonths[i] = str(count)
    count += 1

for i in range(0, 19):
    url = leftUrl + yearAndMonths[i] + rightUrl
    path = leftPath + yearAndMonths[i] + rightPath
    if i == 6 or i == 5:
        url = leftUrl + yearAndMonths[i] + "-citbike-tripdata.csv.zip"
        path = leftPath + yearAndMonths[i] + "-citbike-tripdata.csv.zip"
    urllib.request.urlretrieve(url, path)
    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall("/tmp/")

    currData = spark.read.format("csv").option("header", True).option("inferSchema", True).load("file:" + path[:-4])
    bikeData = bikeData.union(currData)



###Preprocess bikedata 

In [0]:
processedBikeData = bikeData
'''
processedBikeData = processedBikeData.withColumn('year', col('started_at').substr(1,4))\
    .withColumn('month', col('started_at').substr(6,2))\
    .withColumn('day', col('started_at').substr(9,2))\
    .groupBy('year', 'month', 'day')\
    .count()\
    .withColumnRenamed("count", "trip_count")\
    .collect()
'''
processedBikeData = processedBikeData.withColumn('datetime', substring('started_at', 1, 10))\
    .groupBy("datetime")\
    .count()\
    .withColumnRenamed("count", "trip_count")\
    .withColumnRenamed("datetime", "bike_date_time")


###Load weather data from DBFS, preprocess weather data, handle null values

In [0]:
from pyspark.ml.feature import Imputer

imputeColumns = ['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'humidity', 'precip', 'precipprob', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir', 'cloudcover', 'visibility', 'severerisk']
weatherData = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/FileStore/tables/weather_data.csv")
weatherData = weatherData.drop('name', 'sealevelpressure', 'sunrise', 'sunset', 'moonphase', 'stations', 'solarradiation', 'solarenergy', 'uvindex', 'dew')

imputer = Imputer(
    inputCols = imputeColumns, 
    outputCols = imputeColumns
)

weatherData = imputer.setStrategy("median").fit(weatherData).transform(weatherData)\
    .fillna('None')


In [0]:
display(weatherData.summary())

summary,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,cloudcover,visibility,severerisk,conditions,description,icon
count,577,577,577,577,577,577,577,577,577,577,577,577,577,577,577,577,577,577,577,577,577,577
mean,17.663258232235698,10.263778162911624,13.70901213171575,17.369324090121317,8.478509532062391,12.758405545927207,60.564818024263474,1.9972287694974025,50.433275563258235,9.850034662045065,null,0.09064124783362218,0.16187175043327542,50.41715771230504,20.71854419410747,195.76273830155958,42.81767764298093,15.0109185441941,15.320623916811092,null,null,null
stddev,9.589490872764063,9.037148733338443,9.09195435086733,10.664266253051068,11.013220710288948,10.57229589582405,14.697219349504598,5.5906202025944305,50.041505014816344,16.003820702636286,null,0.6410298012907704,1.1299591150033044,17.985075802720363,6.437071395807839,101.5770842423778,31.15112802510022,1.9429007224538704,15.159206302480454,null,null,null
min,-9.3,-14.2,-11.6,-15.8,-26.0,-20.2,24.3,0.0,0,0.0,None,0.0,0.0,13.3,8.0,0.2,0.1,4.1,10,Clear,Becoming cloudy in the afternoon with afternoon rain.,clear-day
25%,10.0,3.4,6.7,9.3,0.1,4.2,49.7,0.0,0,0.0,null,0.0,0.0,35.3,16.2,89.2,14.3,15.1,10,null,null,null
50%,17.9,9.9,13.5,17.9,8.5,13.5,59.6,0.008,100,4.17,null,0.0,0.0,48.9,20.0,233.6,36.6,15.9,10,null,null,null
75%,26.1,18.0,21.6,26.1,18.0,21.6,71.2,0.862,100,12.5,null,0.0,0.0,63.5,24.0,274.4,69.3,16.0,10,null,null,null
max,36.1,26.7,30.7,41.0,29.5,34.7,95.4,60.941,100,100.0,snow,8.1,16.1,96.1,46.4,359.8,100.0,16.0,100,"Snow, Rain, Partially cloudy",Partly cloudy throughout the day.,snow



###Join weather data and bike data and store resulting data in DBFS

In [0]:
result_data = processedBikeData.join(weatherData, processedBikeData.bike_date_time == weatherData.datetime).drop("bike_date_time")
result_data.coalesce(1).write.format('csv').option('header', 'true').mode('overwrite').save("/FileStore/tables/result_data.csv")


###Load resulting data from DBFS into dataframe

In [0]:
result_data = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/FileStore/tables/result_data.csv")


###Further Data Pre-Processing and Filter resulting data into training data and testing data

In [0]:
result_data = result_data.withColumn("week_day_number", date_format(("datetime"), "E"))
  #.drop('humidity', 'precip', 'precipprob', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir', 'cloudcover', 'preciptype', 'conditions', 'description', 'icon')
trainingData = result_data.filter(result_data.datetime.between("2022-01-01", "2022-12-31"))
testingData = result_data.filter(result_data.datetime.between("2023-01-01", "2023-07-31"))

## 2. Build ML Pipeline using Training Data


###Log Scale Trip Count

In [0]:
logTrainingData = trainingData.withColumn("log_trip_count", log(col("trip_count")))
logTestingData = testingData.withColumn("log_trip_count", log(col("trip_count")))


###StringIndexer, OneHotEncoder, Feature Selection with VectorAssembler


In [0]:
#rFormula = RFormula(formula="log_trip_count ~ . - trip_count", featuresCol="features", labelCol="log_trip_count", handleInvalid="skip")

from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import SQLTransformer

categoricalCols = ["week_day_number"]
indexOutputCols = [x + "Index" for x in categoricalCols]
oheOutputCols = [x + "OHE" for x in categoricalCols]

stringIndexer = StringIndexer(inputCols=categoricalCols, 
                              outputCols=indexOutputCols, 
                              handleInvalid="skip")
oheEncoder = OneHotEncoder(inputCols=indexOutputCols, 
                           outputCols=oheOutputCols)

numericCols = ["tempmax", "tempmin", "temp", "feelslikemax", "feelslikemin", "feelslike", "precipcover", "visibility", "severerisk"]
assemblerInputs = oheOutputCols + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs, 
                               outputCol="features")


###Build machine learning model 

In [0]:
lr = LinearRegression(featuresCol="features", labelCol="log_trip_count", predictionCol="log_pred")
stages = [stringIndexer, oheEncoder, vecAssembler, lr]
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(logTrainingData)

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

## 3. Evaluate ML Pipeline using Testing Data

In [0]:
predDF = pipelineModel.transform(logTestingData)


###Exponentiate Trip Count Prediction 

In [0]:
expDF = predDF.withColumn("prediction", exp(col("log_pred")))
regressionEvaluator = RegressionEvaluator(labelCol="trip_count", predictionCol="prediction")


###Evaluation with Mean Absolute Error (MAE) and R Squared Value (R2)

In [0]:
mae = regressionEvaluator.setMetricName("mae").evaluate(expDF)
r2 = regressionEvaluator.setMetricName("r2").evaluate(expDF)
print(f"MAE is {mae}")
print(f"R2 is {r2}")

MAE is 11674.90853940699
R2 is 0.7780053768031745
